### **Singapore Tourism Dashboard**
***
**Project Description:** \
This project explores the creation of an interactive Data Visualisation Dashboard using Tourism Listing data from the Singapore Tourism Board.

> **Singapore Tourism Map:** Locations of Attractions, Accomodation and F&B Listings plotted across the Map of Singapore\
        - Use Case for *Visitors*: Locate tourism hotspots for holiday planning\
        - Use Case for *Tourism Businesses*: Identify geographical opportunities to set up new tourism offerings in Singapore
        \
        \
> **Rating & Pricing Scorecard:** Scatterplots of the Rating & Price Scores and Percentile of Tourism Listings\
        - Use Case for *Visitors*: Compare ratings and prices of all listings and get a sense of the averages\
        - Use Case for *Tourism Businesses*: Benchmarking to compare rating and price performance against other businesses
      
**Contributers:** Lim Yu Bin

**Last Updated On:** 17 Jan 2023

### Import Libraries

In [13]:
## Import Libraries
# For Handling of API & Response
import requests, json

# Store / Retrieve API Key as Environment Variable
import os
from dotenv import load_dotenv

# For Data Processing
import pandas as pd
import numpy as np
from time import sleep

# For REGEX Data Manipulation
import re

# For Dashboarding
import hvplot.pandas
import panel as pn
from bokeh.models import HoverTool

# For Visualisation
import folium as fm

## Display Mode for Visualisation
pn.extension(sizing_mode="stretch_width")

### Data Retrieval
**Tourism Listings Data (Attraction, Accomodation, Food & Beverage)**<br>
Data Source: __[Singapore Tourism Board TIH API](https://tih-dev.stb.gov.sg/content-api/apis)__

In [14]:
# # Note: To retrieve the latest data using the API, 
# #       1) Request for your own TIH API Key
# #       2) Create a .env file in working directory with content: TIH_API_KEY=your_own_api_key

# # Retrive TIH API Key as Environment Variable in .env file
# load_dotenv()
# TIH_API_KEY = os.getenv('TIH_API_KEY')

# # Call the TIH API to retrieve Toursim Listings (Attraction, Accomodation, F&B) in Singapore 
# url = "https://api.stb.gov.sg/content/common/v2/search"
# headers = {"Content-Type": "application/json", "X-Content-Language": "en", "X-API-Key": str(TIH_API_KEY)}
# payload = {"dataset": "accommodation, attractions, food_beverages", "limit" : 50, "offset" : 0, "distinct": "Yes"}
# try:
#     response = requests.get(url, headers = headers, params = payload)
#     if response.status_code == 200:
#         response_json = response.json()['data']
#         place_list = response_json
#     else:
#         print(response.status_code)
# except Exception as e:
#     sleep(5)

# offset_counter = 50

# # Call API to populate dataframe in batches of 50 (max batch size = 50)
# while len(response_json) == 50:
#     try:
#         payload = {"dataset": "accommodation, attractions, food_beverages", "limit" : 50, "offset" : offset_counter, "distinct": "Yes"}
#         response = requests.get(url, headers = headers, params = payload)
#         if response.status_code == 200:
#             response_json = response.json()['data']
#             place_list.extend(response_json)
#             offset_counter += 50
#             print(offset_counter)
#         else:
#             print(response.status_code)
#     except Exception as e:
#         sleep(5)
    
# df = pd.DataFrame(place_list)

In [15]:
# # Store df as pickle to be reused 
# df.to_pickle("./Singapore_Places_Raw.pkl")

# Alternative, read extracted raw data using pickle file (Updated as of 17 Jan 2023)
df = pd.read_pickle("./Singapore_Places_Raw.pkl") 

### Data Exploration & Data Cleaning

In [16]:
## Places (Attraction, Accom, F&B) Table
print(df.shape)
print(df.columns)
pd.set_option('max_columns', None)

## Remove fields not required
df_edit = df.copy()
df_edit = df_edit[['uuid', 'name', 'categoryDescription', 'type', 'rating', 'pricing',  
                'leadInRoomRates','address', 'location']]

(1307, 37)
Index(['uuid', 'name', 'type', 'tags', 'description', 'body', 'rating',
       'location', 'address', 'thumbnails', 'images', 'videos', 'documents',
       'source', 'metadata', 'categoryDescription', 'dataset', 'reviews',
       'pricing', 'companyDisplayName', 'supportedLanguage', 'amenities',
       'businessHour', 'contact', 'nearestMrtStation', 'officialWebsite',
       'officialEmail', 'staYear', 'admissionInfo', 'ticketed', 'group',
       'temporarilyClosed', 'businessHourNotes', 'cuisine', 'leadInRoomRates',
       'leadInRoomSize', 'noOfRooms'],
      dtype='object')


In [17]:
## Exploratory Analysis on Tourism Listings (Attraction, Accom, F&B) Table
print("No. of unique uuid:", df['uuid'].nunique())
#display(df_edit.describe(include='all'))

## List of Type for each Category
#display(pd.DataFrame(df_clean.groupby(['categoryDescription','type']).size()))

## List of Lead in Room Rate
# df_edit['leadInRoomRates'].unique()

No. of unique uuid: 1307


In [18]:
## Processing Listings Table

# 1) Data Cleaning to replace Empty String with NaN
df_edit = df_edit.replace('', np.NaN)
df_edit.loc[df_edit['pricing'] == {'others': ''}, 'pricing'] = np.NaN

# 2) Extract Longitude and Latitude values from Location field for point mapping
pd.options.display.precision = 12
df_edit_copy = df_edit.copy()
df_edit[['latitude','longitude']] = pd.DataFrame(list(df_edit_copy['location']))

# 3) Combine Pricing and LeadInRoomRates into a single Price field
df_edit['price'] = np.where(type(df_edit['pricing']) is dict, np.NaN, df_edit['leadInRoomRates'])

# For Attarctions & F&B Price = Populate with Pricing field, else Lead In Room Rates
for index, dic in enumerate(df_edit['pricing']):
    if df_edit.loc[index, 'categoryDescription'] == "Attractions" or df_edit.loc[index, 'categoryDescription'] == "Food & Beverages":
        if type(df_edit.loc[index, 'pricing']) is dict:
            df_edit.loc[index, 'price'] = dic.get('others')
        elif df_edit.loc[index, 'pricing'] != np.NaN:
            df_edit.loc[index, 'price'] = df_edit.loc[index, 'pricing']
        else:
            df_edit.loc[index, 'price'] = df_edit.loc[index, 'leadInRoomRates']
            
# For Accomodation Price = Populate with Lead In Room Rates field, else Pricing        
    elif df_edit.loc[index, 'categoryDescription'] == "Accommodation":
        if df_edit.loc[index, 'leadInRoomRates'] != np.NaN:
            df_edit.loc[index, 'price'] = df_edit.loc[index, 'leadInRoomRates']
        else:
            df_edit.loc[index, 'price'] = df_edit.loc[index, 'pricing']   

# 4) Add a priceFloor and priceCeiling fields based on Price
df_edit['price'] = df_edit['price'].astype(str)
df_edit['priceFloor'] = np.NaN
df_edit['priceCeiling'] = np.NaN

for index, price in enumerate(df_edit['price']):
    
    price_list = []
    
    if 'free' in str(price).lower():
        price_list.append(0)
    
    # Search the string for <$X
    if re.search(r'<[a-zA-Z]*\$*\d+', price):
        price_list.extend(re.findall(r'\$\d+(?:\.\d+)?', price))
        # If the only $X in the string is <$X, add 0 to price_list
        if len(price_list) == 1:
            price_list.append(0)
            
    # Search the string for $X / $X.XX, put only those digits with $ in price_list
    elif re.search(r'\$\d+', price):
        price_list.extend(re.findall(r'\$\d+(?:\.\d+)?', price))
        
    # Search the string for $ / $$ / $$$ / $$$$, replace $ with estimated price in price_list
    elif re.search(r'\$+', price):
        price_symbol = re.findall(r'\$+', price)
        if '$' in price_symbol:
            price_list.extend([0, 10])
        if '$$' in price_symbol:
            price_list.extend([10, 30])
        if '$$$' in price_symbol:
            price_list.extend([30, 60])
        if '$$$$' in price_symbol:
            price_list.extend([60, 99999])  
    
    # Search the string for X / X.XX, put all digits in price_list
    elif re.search(r'\d+', price):
        price_list.extend(re.findall(r'\d+(?:\.\d+)?', price))
        
    # For those without any price matching above conditions, set price_list = [0, 99999]     
    if len(price_list) == 0:
        price_list.extend([0, 99999])
        
    # Remove the $ in price_list and convert to float
    price_list = [float(str(p).strip('$')) for p in price_list]
    price_list = sorted(price_list, key = float)
    
    # Based on lowest and highest price in price_list, set the priceFloor and priceCeiling
    if len(price_list) == 1:
        df_edit.loc[index, 'priceFloor'] = price_list[0]
        df_edit.loc[index, 'priceCeiling'] = float(99999)
    else:
        df_edit.loc[index, 'priceFloor'] = price_list[0]
        df_edit.loc[index, 'priceCeiling'] = price_list[-1]

# 5) Format Address field 
for index, dic in enumerate(df_edit['address']):
    if type(df_edit.loc[index, 'address']) is dict:
        
        if dic.get('block') == '':
            block = ''
        else:
            block = str(dic.get('block') + " ")
            
        if dic.get('streetName') == '':
            streetName = ''
        else:
            streetName = str(dic.get('streetName') + " ")
            
        if dic.get('floorNumber') == '':
            floorNumber = ''
        else:
            floorNumber = str("#" + dic.get('floorNumber'))
            
        if dic.get('unitNumber') == '':
            unitNumber = ''
        else:
            unitNumber = str("-" + dic.get('unitNumber') + " ")
            
        if dic.get('buildingName') == '':
            buildingName = ''
        else:
            buildingName = str(dic.get('buildingName') + " ")
            
        if dic.get('postalCode') == '':
            postalCode = ''
        else:
            postalCode = str("Singapore " + dic.get('postalCode')) 
            
        df_edit.loc[index, 'fullAddress'] = str(block + streetName + floorNumber + unitNumber + buildingName + postalCode)

# 6) Data Cleaning using Postal Code to map missing Geocoordinates        
df_edit.loc[df_edit['uuid'] == '00257bc1723974446478d717e9df38928e2',['latitude', 'longitude']] = [1.302879, 103.864966]
df_edit.loc[df_edit['uuid'] == '005ac7109c7447147a796288e0e7c181a57',['latitude', 'longitude']] = [1.3050, 103.8331]
df_edit.loc[df_edit['uuid'] == '002f8672d2a0624422f89cd1ce97c726659',['latitude', 'longitude']] = [1.3016, 103.8602]
df_edit.loc[df_edit['uuid'] == '002486d4dac58e04b81b1bbb1c696ee589f',['latitude', 'longitude']] = [1.2847, 103.8324]
df_edit.loc[df_edit['uuid'] == '005999c192a491e4a909b4e58640174a20e',['latitude', 'longitude']] = [1.2996, 103.8552]
df_edit.loc[df_edit['uuid'] == '00280910e9d151a4459890e96fc278b6410',['latitude', 'longitude']] = [1.2556, 1.2556]
df_edit.loc[df_edit['uuid'] == '005884154771b6f48d591a09b23fa72186a',['latitude', 'longitude']] = [1.2897, 103.8513]
df_edit.loc[df_edit['uuid'] == '002c6fc48297a4d4de1993dba7d204d42e1',['latitude', 'longitude']] = [1.3110, 103.8623]
df_edit.loc[df_edit['uuid'] == '0015253b6f670e3443db3f217c226b300f0',['latitude', 'longitude']] = [1.3361, 103.9634]

# 7) Convert "Others" type to 3 distinct types between the 3 categories
df_edit.loc[(df_edit['categoryDescription'] == "Attractions") & (df_edit['type'] == "Others"), 'type'] = "Other Attractions"
df_edit.loc[(df_edit['categoryDescription'] == "Accommodation") & (df_edit['type'] == "Others"), 'type'] = "Other Accommodations"
df_edit.loc[(df_edit['categoryDescription'] == "Food & Beverages") & (df_edit['type'] == "Others"), 'type'] = "Other Food & Beverages"

# 8) Add missing type for The Singing Bowl Gallery (uuid: 002b1ffabd493cf491980a0ee5a0b420521)
df_edit.loc[df_edit['uuid'] == "002b1ffabd493cf491980a0ee5a0b420521", 'type'] = "Fitness and Holistic Wellness"


# 9) Add new PriceLevel field as a fixed price for the Listing
df_edit['priceLevel'] = np.where((df_edit['priceCeiling'] >= df_edit['priceFloor']) & (df_edit['priceCeiling'] != float(99999)), df_edit['priceCeiling'], df_edit['priceFloor'])

df_clean = df_edit[['uuid', 'name', 'categoryDescription', 'type', 'rating', 'price',  
                'priceFloor','priceCeiling', 'priceLevel', 'latitude', 'longitude', 'fullAddress']]

## Export Cleaned Dataset
# df_clean.to_excel("Singapore_Places_Cleaned.xlsx") 

In [19]:
# Display the cleaned dataset
df_clean.head(2)

,uuid,name,categoryDescription,type,rating,price,priceFloor,priceCeiling,priceLevel,latitude,longitude,fullAddress
0,00239f0d56be1f14f5f86b54bbb4508572c,A Minion's Perspective Experience,Attractions,Leisure & Recreation,0.0,from $22,22.0,99999.0,22.0,1.256752,103.820331,8 Sentosa Gateway #01-38/39 Resorts World Sent...
1,0021720728628c94a298cf5f6b12d0d45b2,Chinatown Heritage Centre - Suspended till Fur...,Attractions,History & Culture,0.0,nan,0.0,99999.0,0.0,1.282180,103.844150,"48 Pagoda Street, Singapore 059207"


### Data Visualisation

In [20]:
# Convert dataframe to an interactive dataframe which can be manipulated using Widgets
idf = df_clean.interactive()

In [21]:
# Retrieve the list of unique Type for each catgoryDescription
df_category_type = df_clean.groupby('categoryDescription')['type'].unique().copy()

# Create the Widgets for the dashboard
price_slider = pn.widgets.RangeSlider(name = 'Price Range', start=0.00, end=1000.00, step =0.01, format = '$0,0.00')

rating_slider = pn.widgets.RangeSlider(name = 'Rating Range', start=0.0, end=5.0, step =0.1, format = '0.0')

category_checkbutton = pn.widgets.CheckButtonGroup(name='Category', value=df_category_type.index.tolist(), options=df_category_type.index.tolist())

accom_checkbox = pn.widgets.MultiChoice(name='Accommodation Type', value=df_category_type['Accommodation'].tolist(), options=df_category_type['Accommodation'].tolist(), solid = False)

attraction_checkbox = pn.widgets.MultiChoice(name='Attraction Type', value=df_category_type['Attractions'].tolist(), options=df_category_type['Attractions'].tolist(), solid = False)

food_checkbox = pn.widgets.MultiChoice(name='Food & Beverages Type', value=df_category_type['Food & Beverages'].tolist(), options=df_category_type['Food & Beverages'].tolist(), solid = False)

In [22]:
# get_map: Create a map using Folium with default Latitude and Longitude in the middle of Singapore
def get_map(lat=1.28967, long=103.85007, zoom_start=12):
    return fm.Map(location=[lat,long], zoom_start=zoom_start, tiles='Cartodb Positron')

# get_icon: Configure the Marker icons in the Map Visualisa
def get_icon(categoryDescription):
    if categoryDescription == "Accommodation":
        return fm.Icon(icon='bed',
                       color='cadetblue',
                       icon_color='#FFFFFF',
                       prefix = 'fa'
                       )
    
    elif categoryDescription == "Attractions":
        return fm.Icon(icon='flag',
                       color='lightred',
                       icon_color='#FFFFFF',
                       prefix = 'fa'
                       )
    
    elif categoryDescription == "Food & Beverages":
        return fm.Icon(icon='cutlery',
                       color='blue',
                       icon_color='#FFFFFF',
                       prefix = 'fa'
                       )
    
# @pn.depends will trigger update_map function whenever the Widget values are changed
# update_map: Creation of Map Visualisation with interactive dataframe and Widget
@pn.depends(price_slider = price_slider, rating_slider = rating_slider, category_checkbutton = category_checkbutton, accom_checkbox = accom_checkbox, attraction_checkbox = attraction_checkbox, food_checkbox = food_checkbox)
def update_map(price_slider, rating_slider, category_checkbutton, accom_checkbox, attraction_checkbox, food_checkbox):
    
    # Create Singapore Map
    sg_map = get_map()
    
    # Filter the interactive dataframe based on Column values mapped to Widgets values
    # Issue: Folium Map is unable to add more than 500 Markers of the same categoryDescription at one time, workaround to limit the number of Markers when Restaurants is chosen
    if "Attractions" in category_checkbutton and "Restaurants" in food_checkbox:
        ipipeline = (
            idf[(idf['categoryDescription'].isin(category_checkbutton)) &
            ((idf['type'].isin(accom_checkbox)) |
            (idf['type'].isin(attraction_checkbox)) |
            (idf['type'].isin(food_checkbox))) &
            (idf['rating'] >= rating_slider[0]) &
            (idf['rating'] <= rating_slider[1]) &
            (~((idf['priceCeiling'] < price_slider[0]) |
            (idf['priceFloor'] > price_slider[1])))].head(500)
            )

    else:
        ipipeline = (
            idf[(idf['categoryDescription'].isin(category_checkbutton)) &
            ((idf['type'].isin(accom_checkbox)) |
            (idf['type'].isin(attraction_checkbox)) |
            (idf['type'].isin(food_checkbox))) &
            (idf['rating'] >= rating_slider[0]) &
            (idf['rating'] <= rating_slider[1]) &
            (~((idf['priceCeiling'] < price_slider[0]) |
            (idf['priceFloor'] > price_slider[1])))]
            )

    # For each row in the filtered dataframe, create a Marker on the Map
    # Create the Tooltip (when hovering over Marker) and Pop-up (displayed after clicking on the Marker)
    ipipeline.apply(
        lambda row: fm.Marker(
            location=[row['latitude'], row['longitude']],
            icon=get_icon(row['categoryDescription']),
            popup = fm.Popup('<strong>' + str(row['name']) + '</strong>' + '<br>Category: ' + str(row['categoryDescription']) + '<br>Type: ' + str(row['type']) + '<br>Price: ' + html.escape(row['price']) + '<br>Rating: ' + str(row['rating']) + '<br>Address: ' + str(row['fullAddress']), min_width = 225, max_width = 225),
            tooltip='<strong>' + str(row['name']) + '</strong>' + '<br>Category: ' + str(row['categoryDescription']) + '<br>Type: ' + str(row['type']) + '<br>Price: ' + html.escape(row['price'])
            ).add_to(sg_map),axis=1)
    
    # Place the Folium Map into a Panel pane so that it maybe used in Panel dashboard
    folium_pane = pn.pane.plot.Folium(sg_map, sizing_mode="stretch_both", min_height=500, margin=0)
    
    # Output of the update_map function is the Panel pane with the populated Folium Map
    return folium_pane

# update_scatter_rating: Create a scatter plot based on Rating v.s. Rating Percentile of every Listing
def update_scatter_rating(price_slider, rating_slider, category_checkbutton, accom_checkbox, attraction_checkbox, food_checkbox):
    pipeline = (
        df_clean[(df_clean['categoryDescription'].isin(category_checkbutton)) &
        ((df_clean['type'].isin(accom_checkbox)) |
        (df_clean['type'].isin(attraction_checkbox)) |
        (df_clean['type'].isin(food_checkbox))) &
        (df_clean['rating'] >= rating_slider[0]) &
        (df_clean['rating'] <= rating_slider[1]) &
        (df_clean['priceLevel'] >= price_slider[0]) &
        (df_clean['priceLevel'] <= price_slider[1])]
    )
    ipipeline = pipeline.copy()
    ipipeline['ratingRank'] = ipipeline.loc[:,'rating'].rank(ascending = True, method = "first")
    ipipeline['ratingPercentile'] = ipipeline.loc[:,'rating'].rank(ascending = True, pct = True, method = "first")*100
    
    scatter = ipipeline.hvplot(kind='scatter', x='ratingPercentile', y='rating', by='categoryDescription',hover_cols=["name", "categoryDescription", "type", "rating", "ratingPercentile", "ratingRank"], grid = True, legend = "bottom", xlabel = "Percentile (%)", ylabel = "Rating (Out of 5)", color = ['#426776', '#FF8B7D','#38A9DB'])
    return scatter

# update_scatter_price: Create a scatter plot based on Price v.s. Price Percentile of every Listing
def update_scatter_price(price_slider, rating_slider, category_checkbutton, accom_checkbox, attraction_checkbox, food_checkbox):
    pipeline = (
        df_clean[(df_clean['categoryDescription'].isin(category_checkbutton)) &
        ((df_clean['type'].isin(accom_checkbox)) |
        (df_clean['type'].isin(attraction_checkbox)) |
        (df_clean['type'].isin(food_checkbox))) &
        (df_clean['rating'] >= rating_slider[0]) &
        (df_clean['rating'] <= rating_slider[1]) &
        (df_clean['priceLevel'] >= price_slider[0]) &
        (df_clean['priceLevel'] <= price_slider[1])]
    )
    ipipeline = pipeline.copy()
    ipipeline['priceRank'] = ipipeline.loc[:,'priceLevel'].rank(ascending = True, method = "first")
    ipipeline['pricePercentile'] = ipipeline.loc[:,'priceLevel'].rank(ascending = True, pct = True, method = "first")*100
    
    scatter = ipipeline.hvplot(kind='scatter', x='pricePercentile', y='priceLevel', by='categoryDescription',hover_cols=["name", "categoryDescription", "type", "priceLevel", "pricePercentile", "priceRank", "price"], grid = True, legend = "bottom", xlabel = "Percentile (%)", ylabel = "Price ($)", color = ['#426776', '#FF8B7D','#38A9DB'])
    return scatter

# ## Test Functions to Create Map Visualisation 

# @pn.depends(price_slider = price_slider, rating_slider = rating_slider, category_checkbutton = category_checkbutton, accom_checkbox = accom_checkbox, attraction_checkbox = attraction_checkbox, food_checkbox = food_checkbox)
# def update_map_all(price_slider, rating_slider, category_checkbutton, accom_checkbox, attraction_checkbox, food_checkbox):
    
#     # Create Singapore Map
#     sg_map = get_map()
    
#     # Filter the interactive dataframe based on Column values mapped to Widgets values
#     ipipeline = (
#         idf[(idf['categoryDescription'].isin(category_checkbutton)) &
#         ((idf['type'].isin(accom_checkbox)) |
#         (idf['type'].isin(attraction_checkbox)) |
#         (idf['type'].isin(food_checkbox))) &
#         (idf['rating'] >= rating_slider[0]) &
#         (idf['rating'] <= rating_slider[1]) &
#         (~((idf['priceCeiling'] < price_slider[0]) |
#         (idf['priceFloor'] > price_slider[1])))]
#         )
    
#     # For each row in the filtered dataframe, create a Marker on the Map
#     # Create the Tooltip (when hovering over Marker) and Pop-up (displayed after clicking on the Marker)
#     feature_group = fm.map.FeatureGroup()
    
#     ipipeline.apply(
#         lambda row: fm.Marker(
#             location=[row['latitude'], row['longitude']],
#             icon=get_icon(row['categoryDescription']),
#             popup = fm.Popup('<strong>' + str(row['name']) + '</strong>' + '<br>Category: ' + str(row['categoryDescription']) + '<br>Type: ' + str(row['type']) + '<br>Price: ' + html.escape(row['price']) + '<br>Rating: ' + str(row['rating']) + '<br>Address: ' + str(row['fullAddress']), min_width = 225, max_width = 225),
#             tooltip='<strong>' + str(row['name']) + '</strong>' + '<br>Category: ' + str(row['categoryDescription']) + '<br>Type: ' + str(row['type']) + '<br>Price: ' + html.escape(row['price'])
#             ).add_to(feature_group),axis=1)
        
#     sg_map.add_child(feature_group)
#     # sg_map.add_child(LayerControl())
    
#     # Place the Folium Map into a Panel pane so that it maybe used in Panel dashboard
#     folium_pane = pn.pane.plot.Folium(sg_map, sizing_mode="stretch_both", min_height=500, margin=0)
    
#     # Output of the update_map function is the Panel pane with the populated Folium Map
#     return folium_pane


# @pn.depends(price_slider = price_slider, rating_slider = rating_slider, category_checkbutton = category_checkbutton, accom_checkbox = accom_checkbox, attraction_checkbox = attraction_checkbox, food_checkbox = food_checkbox)
# def update_map_fast(price_slider, rating_slider, category_checkbutton, accom_checkbox, attraction_checkbox, food_checkbox):
    
#     # Create Singapore Map
#     sg_map = get_map()
    
#     # Filter the interactive dataframe based on Column values mapped to Widgets values
#     ipipeline = (
#         idf[(idf['categoryDescription'].isin(category_checkbutton)) &
#         ((idf['type'].isin(accom_checkbox)) |
#         (idf['type'].isin(attraction_checkbox)) |
#         (idf['type'].isin(food_checkbox))) &
#         (idf['rating'] >= rating_slider[0]) &
#         (idf['rating'] <= rating_slider[1]) &
#         (~((idf['priceCeiling'] < price_slider[0]) |
#         (idf['priceFloor'] > price_slider[1])))]
#         )
    
#     # For each row in the filtered dataframe, create a Marker on the Map
#     # Create the Tooltip (when hovering over Marker) and Pop-up (displayed after clicking on the Marker)
#     lat_long = []
#     for i in range(ipipeline.shape[0]):
#         row_list = [ipipeline['latitude'].values[i], ipipeline['longitude'].values[i]]
#         lat_long.append(row_list)
#     sg_map.add_child(FastMarkerCluster(lat_long))    
#     # Place the Folium Map into a Panel pane so that it maybe used in Panel dashboard
#     folium_pane = pn.pane.plot.Folium(sg_map, sizing_mode="stretch_both", min_height=500, margin=0)
    
#     # Output of the update_map function is the Panel pane with the populated Folium Map
#     return folium_pane


# @pn.depends(price_slider = price_slider, rating_slider = rating_slider, category_checkbutton = category_checkbutton, accom_checkbox = accom_checkbox, attraction_checkbox = attraction_checkbox, food_checkbox = food_checkbox)
# def update_map_fast_pkl(price_slider, rating_slider, category_checkbutton, accom_checkbox, attraction_checkbox, food_checkbox):
    
#     # Create Singapore Map
#     sg_map = get_map()
    
#     # Filter the interactive dataframe based on Column values mapped to Widgets values
#     ipipeline = (
#         idf[(idf['categoryDescription'].isin(category_checkbutton)) &
#         ((idf['type'].isin(accom_checkbox)) |
#         (idf['type'].isin(attraction_checkbox)) |
#         (idf['type'].isin(food_checkbox))) &
#         (idf['rating'] >= rating_slider[0]) &
#         (idf['rating'] <= rating_slider[1]) &
#         (~((idf['priceCeiling'] < price_slider[0]) |
#         (idf['priceFloor'] > price_slider[1])))]
#         )
    
#     # For each row in the filtered dataframe, create a Marker on the Map
#     # Create the Tooltip (when hovering over Marker) and Pop-up (displayed after clicking on the Marker)
#     ipipeline.to_pickle("./dummy.pkl")
#     df_upd = pd.read_pickle("./dummy.pkl")
#     callback = """\
#     function (row) {
#         var icon, marker;
#         icon = L.AwesomeMarkers.icon({
#         icon: "map-marker", markerColor: "red"});
#         marker = L.marker(new L.LatLng(row[0], row[1]));
#         marker.setIcon(icon);
#         return marker;
#     };
#     """
#     # Instead of pickel, try:
#     # 1) Use lamba to apply the FastMarkerCluster to map
#     # 2) Convert ipipeline to dataframe using .groupby(['uuid','name', 'categoryDescription', 'type', 'price','priceFloor','priceCeiling', 'latitude', 'longitude', 'fullAddress', 'ratingPercentile', 'ratingRank'])['rating'].mean().to_frame().reset_index()
    
#     sg_map.add_child(FastMarkerCluster(df_upd[['latitude','longitude']].values.tolist(), callback=callback))
  
#     # Place the Folium Map into a Panel pane so that it maybe used in Panel dashboard
#     folium_pane = pn.pane.plot.Folium(sg_map, sizing_mode="stretch_both", min_height=500, margin=0)
    
#     # Output of the update_map function is the Panel pane with the populated Folium Map
#     return folium_pane

In [23]:
## Bind the widgets to the Scatter Plots to make the scatterplot interactive with widgets
rating_scatter = pn.bind(update_scatter_rating, price_slider, rating_slider, category_checkbutton, accom_checkbox, attraction_checkbox, food_checkbox)
price_scatter = pn.bind(update_scatter_price, price_slider, rating_slider, category_checkbutton, accom_checkbox, attraction_checkbox, food_checkbox)

## Display Map and the Widgets in IDE
map_viz = pn.Column(price_slider, rating_slider, category_checkbutton, accom_checkbox, attraction_checkbox, food_checkbox, update_map, rating_scatter, price_scatter)
map_viz


Column(sizing_mode='stretch_width')
    [0] RangeSlider(end=1000.0, format='$0,0.00', name='Price Range', sizing_mode='stretch_width', step=0.01, value=(0.0, 1000.0), value_end=1000.0)
    [1] RangeSlider(end=5.0, format='0.0', name='Rating Range', sizing_mode='stretch_width', value=(0.0, 5.0), value_end=5.0)
    [2] CheckButtonGroup(name='Category', options=['Accommodation', ...], sizing_mode='stretch_width', value=['Accommodation', ...])
    [3] MultiChoice(name='Accommodation Type', options=['Serviced Apartments', ...], sizing_mode='stretch_width', solid=False, value=['Serviced Apartments', ...])
    [4] MultiChoice(name='Attraction Type', options=['Leisure & Recreation', ...], sizing_mode='stretch_width', solid=False, value=['Leisure & Recreation', ...])
    [5] MultiChoice(name='Food & Beverages Type', options=['Restaurants', ...], sizing_mode='stretch_width', solid=False, value=['Restaurants', ...])
    [6] ParamFunction(function, sizing_mode='stretch_width')
    [7] ParamFunction(function, sizing_mode='stretch_width')
    [8] ParamFunction(function, sizing_mode='stretch_width')

In [24]:
# Apply the Visualisations and Widgets on a Dashboard Template (using Panel Material template)
material = pn.template.MaterialTemplate(title='Singapore Tourism Dashboard', sidebar_width = 400, header_background = 'rgb(101,168,196)')

# Sidemenu of Dashboard: Contains the Filter Widgets
material.sidebar.append(pn.Column("#Filters",
                                  price_slider,
                                  rating_slider,
                                  "Category",
                                  category_checkbutton,
                                  accom_checkbox,
                                  attraction_checkbox,
                                  food_checkbox))

# Main Display Area: Contains the Map and Scatter Plot visualisations
material.main.append(
    pn.Column(pn.Card(update_map, title = 'Singapore Tourism Map'),
           pn.Row(pn.Card(rating_scatter, title = 'Rating Scorecard'), pn.Card(price_scatter, title = 'Pricing Scorecard'))))
material.servable();

# To display Dashboard on localhost, run in Terminal (Command Prompt):panel serve Singapore_Tourism_Map_Visualisation.ipynb --port 5006
# Change the port (e.g. --port 5007) if the port is already being utilised 

### References & Credits 
- __[Panel - Folium](https://panel.holoviz.org/gallery/external/Folium.html)__
- __[Thu Vu data analytics - Panel/Hvplot Tutorial](https://www.youtube.com/watch?v=uhxiXOTKzfs)__